In [1]:
import data
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd

import librosa
import librosa.display

from IPython.display import Audio

import os

from random import shuffle

In [2]:
path_ds = '/home/edoardobucheli/Datasets/SpeechCommands_v0.02/'

part = 'train'
batch_size = 128
transformation = 'mag'
normalize=True
sr = 16000
len_sec = 1
frame_length=512
frame_step=256
fft_length=512
n_mels=40
n_mfcc=40
mel_lower_edge=125
mel_upper_edge=8000

### Make Dataset

In [3]:
tr_data = data.create_and_batch_dataset_TFSSC(path_ds,
                                              part = part,
                                              batch_size = batch_size,
                                              transformation = transformation,
                                              normalize = normalize,
                                              sr=sr,
                                              len_sec=len_sec,
                                              frame_length=frame_length,
                                              frame_step=frame_step,
                                              fft_length=fft_length,
                                              n_mels=n_mels,
                                              n_mfcc=n_mfcc,
                                              mel_lower_edge=mel_lower_edge,
                                              mel_upper_edge=mel_upper_edge)

Found 36248 files in partition
Instructions for updating:
This will be deleted and should not be used.


### Define Iterator

In [4]:
iterator = tr_data.make_initializable_iterator()
next_step = iterator.get_next()

### Load Model

In [5]:
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Activation, CuDNNGRU
from tensorflow.keras.layers import GRU, LSTM, TimeDistributed, Lambda, Dot, Softmax
from tensorflow.keras.layers import Conv2D, Conv1D, Reshape, Permute, GRUCell, LSTMCell
from tensorflow.keras.layers import Bidirectional, CuDNNLSTM, BatchNormalization, MaxPool1D
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.models import Model
from tensorflow.keras.backend import squeeze,stack, expand_dims

In [6]:
def CRNN_v2(input_shape, n_classes):

    X_input = Input(input_shape)

    X = Permute((2,3,1))(X_input)
    #X = Lambda(lambda q: expand_dims(q, 1), name='expand_dims') (X_input)

    #X = Lambda(lambda q: squeeze(q, 1), name='squeeze_last_dim') (X)
    
    X = Conv2D(32, kernel_size = [5,5], strides = [2,2],
               activation = relu)(X)

    X = Conv2D(1,kernel_size=[1,1], strides = [1,1],
               activation = relu)(X)

    X = Lambda(lambda q: squeeze(q, -1), name='squeeze_last_dim') (X)
    
    X = Bidirectional(CuDNNLSTM(32,return_sequences=True))(X)

    X = Bidirectional(CuDNNLSTM(32, return_sequences = True))(X)

    X = Flatten()(X)

    X = Dense(64)(X)
    X = Dropout(0.5)(X)
    X = Activation(relu)(X)

    X = Dense(n_classes, activation = None)(X)

    model = Model(inputs = X_input, outputs = X)

    return model

In [7]:
#from RNNetworks import CRNN_v2

In [8]:
crnn = CRNN_v2((1,61,257),11)

In [9]:
crnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 61, 257)        0         
_________________________________________________________________
permute (Permute)            (None, 61, 257, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 29, 127, 32)       832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 127, 1)        33        
_________________________________________________________________
squeeze_last_dim (Lambda)    (None, 29, 127)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 29, 64)            41216     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 29, 64)            25088     
__________

In [10]:
x_audio = tf.placeholder(dtype = tf.float32, shape = [None,1,61,257])
y = tf.placeholder(dtype = tf.int32, shape = [None,])

In [11]:
#sounds, labels = next_step
labels_oh = tf.one_hot(y,depth = 11,on_value=1,off_value=0)

logits = crnn(x_audio)
y_hat = tf.argmax(tf.nn.softmax(logits),axis = -1,output_type=tf.int32)

check = tf.cast(tf.equal(y_hat,y),dtype = tf.float32)

acc = tf.reduce_mean(check)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = labels_oh,logits=logits)
optimizer = tf.train.AdamOptimizer()

train_stp = optimizer.minimize(loss)

In [12]:
steps_per_epoch = 36248//batch_size

In [14]:
with tf.Session() as sess:
    sess.run([iterator.initializer,tf.global_variables_initializer()])
    try:
        i = 0
        while True:
            
            [this_batch,this_label] = sess.run(next_step)
            
            
            
    except tf.errors.OutOfRangeError:
        print('Done!')

In [15]:
sess = tf.Session()
sess.run([tf.global_variables_initializer(),iterator.initializer])

[None, None]

In [16]:
steps_per_epoch = 36248//batch_size
epochs = 1

In [17]:
for epoch in range(epochs):
    for i in range(steps_per_epoch):
        this_audio,this_labels = sess.run(next_step)
        _ = sess.run([train_stp,], feed_dict = {x_audio:this_audio,y:this_labels})
        print(i)

0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 